# RAG demo level 3

In [24]:
questions = [
    "What movies are about Abby?",
    "I have seen all Star Wars movies and would like tips for something similar I can watch next.",
    "What is the most common genre of the movies where one of key figures is called Mark?",
    "Are there any movies where Prague takes major role and present city as mysterious and ancient?",
    "When movies about drugs are concerned, is it usually rather serious or funny?",
    "What are some movies featuring a strong female lead that also involve adventure?",
    "Which Western films feature outlaws riding to their doom in the American Southwest?"
]

In [25]:
import subprocess
import os
import json

original_dir = os.getcwd()
try:
    # Jump into the terraform directory
    os.chdir('terraform')

    # Get the database connection string
    SQL_DATABASE = subprocess.run(['terraform', 'output', '-raw', 'SQL_DATABASE'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_USERNAME = subprocess.run(['terraform', 'output', '-raw', 'SQL_USERNAME'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_PASSWORD = subprocess.run(['terraform', 'output', '-raw', 'SQL_PASSWORD'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    SQL_SERVER_FQDN = subprocess.run(['terraform', 'output', '-raw', 'SQL_SERVER_FQDN'], stdout=subprocess.PIPE).stdout.decode('utf-8')

    # Get the embedding model endpoint and key
    model_configurations = subprocess.run(['terraform', 'output', '-raw', 'model_configurations'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    model_config = json.loads(model_configurations)
    embedding_model = model_config["models"]["text-embedding-3-large"]
    EMBEDDINGS_ENDPOINT = embedding_model["endpoint"]
    EMBEDDINGS_KEY = embedding_model["key"]
    gpt_4o_mini_model = model_config["models"]["gpt-4o-mini"]
    GPT_4O_MINI_ENDPOINT = gpt_4o_mini_model["endpoint"]
    GPT_4O_MINI_KEY = gpt_4o_mini_model["key"]
    gpt_4o_model = model_config["models"]["gpt-4o"]
    GPT_4O_ENDPOINT = gpt_4o_model["endpoint"]
    GPT_4O_KEY = gpt_4o_model["key"]

    print(f"Using {SQL_SERVER_FQDN} as the database server")
    print(f"Using {EMBEDDINGS_ENDPOINT} as the embedding model endpoint")
    print(f"Using {GPT_4O_MINI_ENDPOINT} as the gpt-4o-mini model endpoint")
    print(f"Using {GPT_4O_ENDPOINT} as the gpt-4o model endpoint")

finally:
    os.chdir(original_dir)

Using sql-graphrag-psbv.database.windows.net as the database server
Using https://graphrag-psbv.openai.azure.com/ as the embedding model endpoint
Using https://graphrag-psbv.openai.azure.com/ as the gpt-4o-mini model endpoint
Using https://graphrag-psbv.openai.azure.com/ as the gpt-4o model endpoint


In [73]:
from openai import AzureOpenAI 
import pandas as pd
import pyodbc

# Azure SQL Entra login option
# conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SQL_SERVER_FQDN};DATABASE={SQL_DATABASE};UID=tokubica@microsoft.com;Authentication=ActiveDirectoryInteractive;Encrypt=yes;"

# Azure SQL password login option
conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SQL_SERVER_FQDN};DATABASE={SQL_DATABASE};UID={SQL_USERNAME};PWD={SQL_PASSWORD};Encrypt=yes;Charset=Latin1;"

conn = pyodbc.connect(conn_str)

gpt_embedding_client = AzureOpenAI(
    azure_endpoint=EMBEDDINGS_ENDPOINT,
    api_key=EMBEDDINGS_KEY,
    api_version="2025-02-01-preview",
)

gpt_4o_client = AzureOpenAI(  
    azure_endpoint=GPT_4O_ENDPOINT,  
    api_key=GPT_4O_KEY,  
    api_version="2024-08-01-preview",
)

gpt_4o_mini_client = AzureOpenAI(
    azure_endpoint=GPT_4O_MINI_ENDPOINT,  
    api_key=GPT_4O_MINI_KEY,  
    api_version="2024-08-01-preview",
)

In [86]:
from enum import Enum
from pydantic import BaseModel, ValidationError, conlist, Field
from typing import List
import jinja2
import time

class SubQuestions(BaseModel):
    questions: List[str] = Field(description="List of specific sub-questions derived from the main user query, preferably 3 or more")

class FixedQuery(BaseModel):
    query: str = Field(description="Query to be executed in the database, pure SQL with no additional comments. All text-based queries should be hardcoded, just vector will be referenced as %q.")

class BestQuery(BaseModel):
    query: str = Field(description="Query to be executed in the database, pure SQL with no additional comments. All text-based queries should be hardcoded, just vector will be referenced as %q."),
    message_to_user: str = Field(description="Message to be displayed to the user, explaining the purpose of this query and why it was selected as best performing.")

class QueryStrategy(BaseModel):
    id: str = Field(
        description="Unique random identifier for this strategy"
    )
    message_to_user: str = Field(
        description="Message to be displayed to the user, explaining the purpose of this query. Include type of each query step and its relation to other steps. Wording should be about what you are doing, example: 'Searching Movie using keyword search for Star Wars followed by graph query to get related genres and semantic movie search among results."
    )
    query: str = Field(
        description="Rewrite query to better match knowledge base using semantic search. This field is used to create embedding and store as @q variable for command to use."
    )
    command: str = Field(
        description="Command to be executed in the database. This should be a SQL query that can be executed directly against the database. All text-based queries should be hardcoded, just vector will be referenced as %q."
    )

class QueryStrategies(BaseModel):
    strategies: List[QueryStrategy] = Field(
        description="List of strategies, each containing a series of steps to be executed sequentially. Each strategy is a sequence of steps, where each step depends on the results of previous steps."
    )

def get_subquestions(llm_client: AzureOpenAI, question: str) -> SubQuestions:
    with open("./prompts/subquestions.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render()
    messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": question},
        ]
    completion = llm_client.beta.chat.completions.parse(  
        model="gpt-4o",
        messages=messages,
        response_format=SubQuestions,
        max_tokens=1000,  
        temperature=0.7,
    )
    return SubQuestions.model_validate_json(completion.choices[0].message.content)

def prepare_strategies(llm_client: AzureOpenAI, question: str) -> QueryStrategies:
    with open("./prompts/query_strategy_sql.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render()
    messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": question},
        ]
    completion = llm_client.beta.chat.completions.parse(  
        model="gpt-4o",
        messages=messages,
        response_format=QueryStrategies,
        max_tokens=1000,  
        temperature=0.7,
    )
    return QueryStrategies.model_validate_json(completion.choices[0].message.content)

def batch_prepare_strategies(llm_client: AzureOpenAI, questions: SubQuestions) -> QueryStrategies:
    all_strategies = []
    for question in questions.questions:
        strategies = prepare_strategies(llm_client, question)
        all_strategies.extend(strategies.strategies)

    return QueryStrategies(strategies=all_strategies)

def fix_query(llm_client: AzureOpenAI, query: str, error: str):
    with open("./prompts/fix_query.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render(query=query, error=error)
    messages = [
        {"role": "system", "content": system_prompt}, 
        ]
    completion = llm_client.beta.chat.completions.parse(  
        model="gpt-4o",
        messages=messages,
        response_format=FixedQuery,
        max_tokens=1000,  
        temperature=0.7,
    )
    return FixedQuery.model_validate_json(completion.choices[0].message.content).query

def select_best_strategy(llm_client: AzureOpenAI, query: str, executions):
    with open("./prompts/select_best_strategy.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render()
    messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": json.dumps(executions)},
        {"role": "user", "content": query},
        ]
    completion = llm_client.beta.chat.completions.parse(  
        model="gpt-4o",
        messages=messages,
        response_format=BestQuery,
        max_tokens=1000,  
        temperature=0.7,
    )
    return BestQuery.model_validate_json(completion.choices[0].message.content)

def get_embeddings(texts):
    for attempt in range(1, 20):
        try:
            result = gpt_embedding_client.embeddings.create(
                input=texts,
                model="text-embedding-3-large",
                dimensions=1998,
            )
            break
        except Exception as e:
            if hasattr(e, "status_code") and e.status_code == 429:
                time.sleep(10)
            else:
                print(f"Embedding generation error: {e}")
                raise e
    return [json.dumps(item.embedding) for item in result.data]

def database_search(conn, query: str, command: str):
    query_embedding = get_embeddings([query])[0]
    cursor = conn.cursor()
    final_command = f"""
    DECLARE @q VECTOR(1998) = '{query_embedding}';
    {command}
    """
    
    try:
        cursor.execute(final_command)
        rows = cursor.fetchall()
    except Exception as e:
        return {"data": None, "error": e}

    columns = [column[0] for column in cursor.description]
    result = [dict(zip(columns, row)) for row in rows]
    
    return {"data": result, "error": None}

def llm_final_answer(question: str, context_jsons: list[str]) -> str:
    with open("./prompts/query.jinja2", 'r') as f:
        system_prompt_template = jinja2.Template(f.read())
    system_prompt = system_prompt_template.render()
    messages = [{"role": "system", "content": system_prompt}]
    for context_json in context_jsons:
        messages.append({"role": "user", "content": context_json})
    messages.append({"role": "user", "content": question})

    for attempt in range(1, 20):
        try:
            completion = gpt_4o_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                max_tokens=1000,
                temperature=0.7,
            )
            break
        except Exception as e:
            if hasattr(e, "status_code") and e.status_code == 429:
                print(f"LLM response rate limited. Retrying in {10} seconds (Attempt {attempt}/{20})")
                time.sleep(10)
            else:
                print(f"LLM response error: {e}")
                raise e
    else:
        raise Exception("Failed to get LLM final answer after maximum retries.")
    
    return completion.choices[0].message.content

In [87]:
from IPython.display import display, Markdown

for question in questions:
    print(f"\n\n*****************************************************\nQuestion: {question}")

    # Generate sub-questions (reformulations)
    print("Generating sub-questions.")
    sub_questions = get_subquestions(gpt_4o_client, question)
    for sub_question in sub_questions.questions:
        print(f" -> Sub-question: {sub_question}")

    # Generate and execute query strategies
    print("Generating query strategies.")
    strategies = batch_prepare_strategies(gpt_4o_client, sub_questions)
    context_jsons = []
    executions = strategies.model_dump()["strategies"]
    for i, strategy in enumerate(strategies.strategies):
        print(f" -> {strategy.message_to_user}")
        results = database_search(conn, strategy.query, strategy.command)
        if not results["data"]:
            print(f"    -> Trying to fix error in query")
            for attempt in range(1,5):
                fixed_query = fix_query(gpt_4o_client, strategy.query, results["error"])
                results = database_search(conn, strategy.query, fixed_query)
                if results["data"]:
                    print(f"    -> Fixed query successful on attempt {attempt}.")
                    executions[i]["command"] = fixed_query
                    break
            if not results["data"]:
                print("    -> Query fix attempts failed. Moving to next strategy.")
                # executions.strategies[i].results = []
                continue  # move on to the next strategy if no data was found
        context_json = json.dumps(results["data"])
        executions[i]["results"] = results["data"]
        print(f"    -> Found {len(results["data"])} records")
        if context_json != []:
            context_jsons.append(context_json)
    
    # Select the best strategy based on the results
    print("Selecting the best strategy based on the results.")
    best_strategy = select_best_strategy(gpt_4o_client, question, executions)
    print("Expanding results of the best strategy.")
    print(f" -> {best_strategy.message_to_user}")
    results = database_search(conn, question, best_strategy.query)
    print(f"    -> Found {len(results["data"])} records")
    if results["data"]:
        context_json = json.dumps(results["data"])
        context_jsons.append(context_json)

    # Generate the final answer using the LLM
    llm_response = llm_final_answer(question, context_jsons)
    display(Markdown(llm_response))




*****************************************************
Question: What movies are about Abby?
Generating sub-questions.
 -> Sub-question: Which movies feature a character named Abby?
 -> Sub-question: Are there any movies with 'Abby' in the title?
 -> Sub-question: What are famous movies where the main character is called Abby?
Generating query strategies.
 -> Searching for movies featuring a character named Abby using keyword search on Character followed by graph search to retrieve related movies.
    -> Found 18 records
 -> Using semantic search on Character to find entries related to Abby, then retrieving associated movies via graph search.
    -> Found 20 records
 -> Performing a keyword search on Movie to locate entries where the title contains 'Abby'.
    -> Trying to fix error in query
    -> Fixed query successful on attempt 1.
    -> Found 10 records
 -> Conducting a semantic search on Movie to find entries related to 'Abby' in the title.
    -> Found 20 records
 -> Using sema

c:\git\azure-workshops\d-ai-rag\.venv\Lib\site-packages\pydantic\json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Query to be executed in the database, pure SQL with no additional comments. All text-based queries should be hardcoded, just vector will be referenced as %q.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


Expanding results of the best strategy.
 -> This query searches for movies featuring characters named Abby. It was selected because it provides a comprehensive list of movies directly related to your query.
    -> Found 18 records


The following movies prominently feature characters named Abby, each with unique storylines and archetypes:

1. **Happiest Season**: Abby is the central character, a young woman planning to propose to her girlfriend during a family holiday party. Her plans are disrupted when she discovers her partner hasn't yet come out to her conservative parents. This movie explores themes of identity, love, and acceptance.

2. **Mean Girls 2**: Abby (played by Jennifer Stone) is a timid high school student preyed upon by the queen bee Mandi and her clique, The Plastics. Abby becomes a focal point of a fierce girl-world rivalry, with confident senior Jo taking her side. This movie delves into themes of bullying and high school drama.

3. **Nurse 3-D**: Abby Russell is a nurse with a sinister secret life. She targets and punishes dishonest men, embodying a complex archetype of beauty, charisma, and vigilant justice. This thriller explores themes of morality and vengeance.

4. **16 Wishes**: Abby Jensen is a teenage girl who eagerly awaits her 16th birthday to fulfill her wish list. When magical birthday candles arrive, her wishes come true, but one wish threatens to change everything. This movie focuses on themes of adolescence, self-discovery, and magical realism.

These films showcase a variety of Abby archetypes, ranging from romantic leads and high school underdogs to morally complex vigilantes and dreamers. Each story offers unique insights into the character's challenges and growth.



*****************************************************
Question: I have seen all Star Wars movies and would like tips for something similar I can watch next.
Generating sub-questions.
 -> Sub-question: What are some science fiction movies similar to Star Wars?
 -> Sub-question: Can you suggest movies or shows with epic space battles and adventures?
 -> Sub-question: What are some recommendations for movies with themes of rebellion against tyranny?
 -> Sub-question: Are there any film series with iconic characters and expansive universes like Star Wars?
 -> Sub-question: What are other popular franchises that mix science fiction and fantasy elements?
Generating query strategies.
 -> Using semantic search to find science fiction movies similar to Star Wars.
    -> Found 20 records
 -> Performing keyword search for science fiction movies and Star Wars, followed by semantic search within results.
    -> Trying to fix error in query
    -> Fixed query successful on attempt 1.
    -> Found 

...

Movies about drugs can range from serious explorations of the consequences and complexities of drug use and trafficking to humorous depictions that use the theme for comedic effect. The tone often depends on the genre and thematic focus.

### Serious Depictions:
Films that explore the **serious side** of drugs often delve into the moral, societal, and psychological ramifications of drug addiction, trafficking, and the impact on individuals and communities. These movies tend to focus on crime, addiction, struggles, and redemption, offering intense and dramatic portrayals. Examples include:
- **Traffic**: Examines the war on drugs from multiple perspectives, including law enforcement, families, and cartels, highlighting the devastating consequences of drug use and trafficking.
- **Blow**: Chronicles the rise and fall of a drug kingpin, focusing on the allure and dangers of wealth and power in the drug trade.
- **Requiem for a Dream**: Offers a haunting look at addiction, showing the tragic consequences of substance abuse on the lives of the characters.
- **Trainspotting**: Though it contains moments of dark humor, its depiction of heroin addiction is deeply poignant and tragic.

### Funny Depictions:
On the other hand, movies about drugs can also be positioned within the **comedic realm**, using the subject for humor, absurdity, or satire. These films often feature stoner comedy, parody, or exaggerated scenarios surrounding drug use or trafficking. Examples include:
- **Up in Smoke**: A classic stoner comedy that follows two pot-smoking slackers on their misadventures.
- **Half Baked**: A light-hearted comedy about friends trying to bail their friend out of jail, filled with humorous depictions of drug culture.
- **How High**: Combines stoner humor with a fantastical premise, where two characters use "magical" weed to ace their exams and attend Harvard.
- **We're the Millers**: A comedy about a pot dealer who creates a fake family to smuggle drugs across the border, blending humor with crime.

### Mixed Approaches:
Some films blend humor with serious undertones, offering a mix of lightheartedness and reflection:
- **The 51st State**: Uses humor and action to showcase the complexities of drug deals gone wrong.
- **Keanu**: Features comedic characters posing as drug dealers in a crime-driven plot, combining humor with action.

### Conclusion:
The portrayal of drugs in cinema can be **serious** when focusing on addiction, crime, and societal consequences, or **funny** when leaning into stoner comedies, absurd scenarios, or satirical takes on drug culture. The choice depends on the narrative and genre, allowing filmmakers to either provoke thought or provide entertainment.



*****************************************************
Question: What are some movies featuring a strong female lead that also involve adventure?
Generating sub-questions.
 -> Sub-question: What are some movies that feature a strong female lead character?
 -> Sub-question: Which movies involve adventure as a key theme?
 -> Sub-question: Can you list movies that combine a strong female lead with an adventurous plot?
Generating query strategies.
 -> Searching Character using semantic search for 'strong female lead', followed by graph query to find related Movies, then semantic movie search among results.
    -> Found 10 records
 -> Searching Movie using semantic search directly with the query for strong female lead characters.
    -> Found 20 records
 -> Searching for movies featuring the 'Adventure' theme using semantic search.
    -> Found 20 records
 -> Using keyword search to locate movies explicitly linked to the 'Adventure' theme.
    -> Found 20 records
 -> Performing semantic se

Here are some movies that feature a strong female lead and involve adventure:

1. **Tomb Raider**  
   Lara Croft, the fiercely independent daughter of a missing adventurer, must push herself beyond her limits when she finds herself on the island where her father disappeared. This film showcases Lara as a resilient and capable adventurer unraveling mysteries and facing dangers head-on.

2. **Lara Croft: Tomb Raider**  
   English aristocrat Lara Croft, skilled in hand-to-hand combat and archaeology, battles a secret society while recovering lost antiquities. Her strength and intelligence shine as she embarks on thrilling expeditions.

3. **Lara Croft: Tomb Raider - The Cradle of Life**  
   Lara ventures to an underwater temple to retrieve Pandora’s Box. Her determination and resourcefulness are tested as she faces a crime syndicate with catastrophic intentions.

4. **Annihilation**  
   A biologist joins a dangerous expedition into a mysterious zone where the laws of nature don't apply. The female protagonist leads a team through an eerie and challenging environment, showcasing her courage and intellect.

5. **The Assassin**  
   Set during the Tang Dynasty, a female assassin begins questioning her loyalties when she falls in love with one of her targets. The film combines historical intrigue with personal and moral dilemmas.

6. **Kung Fu Yoga**  
   Two professors team up to locate a lost treasure, embarking on an adventure through Tibetan ice caves, Dubai, and mountain temples in India. The female lead contributes significantly to the treasure hunt with her intellect and action skills.

7. **Blade: Trinity**  
   Abigail, a deftly trained vampire hunter, teams up with Blade and Hannibal King to confront powerful foes, including Dracula. Her combat prowess and strategic thinking are pivotal to the group's mission.

8. **Bitch Slap**  
   Three strong-willed women—a stripper, a killer, and a corporate powerbroker—arrive at a desert hideaway to steal $200 million in diamonds. Their bold personalities and adventurous exploits drive the narrative.

9. **Level 16**  
   Teenage girls in an academy uncover the sinister truth behind their training. The film centers on Vivien’s determination to unravel mysteries and fight back, blending suspense with adventure.

10. **Miss Bala**  
    Gloria discovers her inner strength when she is drawn into a dangerous world of cross-border crime. This film combines action and intrigue as Gloria battles adversity and navigates perilous situations.

These films highlight strong female characters who embark on thrilling adventures, showcasing their resilience, intelligence, and bravery.



*****************************************************
Question: Which Western films feature outlaws riding to their doom in the American Southwest?
Generating sub-questions.
 -> Sub-question: What are notable Western films set in the American Southwest?
 -> Sub-question: Which Western films involve outlaw characters as central figures?
 -> Sub-question: Which Western films focus on themes of outlaws riding to their doom?
 -> Sub-question: Are there any iconic Western films depicting outlaws' fate in the American Southwest?
 -> Sub-question: What are examples of Western films with tragic endings for outlaw characters?
Generating query strategies.
 -> Searching Setting using keywords for 'American Southwest' followed by graph query to find related Movies and semantic search among results.
    -> Found 10 records
 -> Performing semantic search on Movie directly for notable Western films set in the American Southwest.
    -> Found 20 records
 -> Searching for Western films using keyword 

Several Western films feature outlaws riding to their doom in the American Southwest, portraying their struggles, final acts of defiance, or ultimate demise. Here are some notable examples:

1. **The Wild Bunch**:
   Aging outlaw Pike Bishop and his gang prepare for one last robbery but are betrayed. They retreat to Mexico, where they face relentless pursuit from a former partner turned bounty hunter. The film culminates in a fierce and bloody showdown, marking their inevitable doom.

2. **The Outlaw Josey Wales**:
   After avenging his family's murder, Josey Wales is hunted by soldiers. Although he prefers traveling alone, he gathers a group of misfits who depend on him. The setting and themes align with the American Southwest, as Wales confronts his fate while protecting those around him.

3. **Butch Cassidy and the Sundance Kid**:
   Butch Cassidy and the Sundance Kid, leaders of the Hole in the Wall Gang, flee to South America to escape the law. Their journey through the rapidly changing American Southwest highlights their doomed fate as outlaws in a world that no longer tolerates their ways.

4. **Once Upon a Time in the West**:
   Although primarily centered on the construction of the railroad, the film features characters such as Cheyenne, an outlaw who meets his end in the rugged terrain of the Southwest. The film explores themes of changing landscapes and the demise of the old ways.

5. **The Assassination of Jesse James by the Coward Robert Ford**:
   Set against the backdrop of the American Southwest, this film explores the mythology surrounding Jesse James, an outlaw whose life and eventual demise are tied to the region's rugged landscapes and lawless reputation.

6. **Young Guns**:
   Led by Billy the Kid, a group of young gunmen seeks vengeance for their benefactor's murder. Their journey in the Southwest ultimately leads them to become the hunted, riding toward their inevitable fate.

These films share a thematic focus on outlaws navigating the harsh realities of the American Southwest, often leading to their tragic end.